In [1]:
import numpy as np

from src.shared import get_data_from_api
import pandas as pd

In [2]:
raw_data = get_data_from_api()
raw_df  = pd.json_normalize(raw_data["chargingStations"])

raw_df.head(5)

,id,name,address,description,totalConnectors,amenities,location.lat,location.lng,connectionsTypes.CCS,connectionsTypes.AC Type 2,connectionsTypes.CHAdeMO
0,B8S7YSLRUBVUO,Agerbæk Børnehus,Debelvej 25,Parkering ved børnehuset.,4,[],55.603490,8.811343,[],"[{'id': 'AGB1', 'status': 'AVAILABLE', 'effect...",[]
1,B9OXSNAF20Z5S,Agerbæk Torv,Fåborgvej 177,None,4,"[SHOPPING, CAFE]",55.600484,8.802200,"[{'id': 'ABT1-A', 'status': 'AVAILABLE', 'effe...",[],[]
2,Alnasenter,Alna Senter,Strømsveien 245,Lynladere i 1 etg utendørs parkeringshus ved M...,17,"[RESTROOM, SHOPPING, CAFE]",59.926581,10.849327,"[{'id': 'AL1-1', 'status': 'AVAILABLE', 'effec...","[{'id': 'AL10', 'status': 'AVAILABLE', 'effect...","[{'id': 'AL1-2', 'status': 'UNAVAILABLE', 'eff..."
3,B8GEWDTNZ8LXC,Alstad Kro og Camping,Alstad 1,None,8,"[RESTROOM, REST_STOP, RESTAURANT, ACCOMMODATIO...",68.272286,13.936212,"[{'id': 'ALC1-1', 'status': 'AVAILABLE', 'effe...","[{'id': 'ALC3-1', 'status': 'AVAILABLE', 'effe...","[{'id': 'ALC1-2', 'status': 'AVAILABLE', 'effe..."
4,BA3I6G2IEG4CG,Altdorf,Prackenfelser Str. 8,None,8,"[RESTROOM, RESTAURANT, SHOPPING, WIFI, CAFE, C...",49.381735,11.344057,"[{'id': 'ALT1-A', 'status': 'AVAILABLE', 'effe...",[],[]


In [3]:
charging_data = raw_df[
        ['id', 'name', 'address', 'description', 'location.lat', 'location.lng', 'amenities']].copy()

charging_data = charging_data.rename({"location.lat": "latitud", "location.lng": "longitud"}, axis=1)

charging_data.head(5)

,id,name,address,description,latitud,longitud,amenities
0,B8S7YSLRUBVUO,Agerbæk Børnehus,Debelvej 25,Parkering ved børnehuset.,55.603490,8.811343,[]
1,B9OXSNAF20Z5S,Agerbæk Torv,Fåborgvej 177,None,55.600484,8.802200,"[SHOPPING, CAFE]"
2,Alnasenter,Alna Senter,Strømsveien 245,Lynladere i 1 etg utendørs parkeringshus ved M...,59.926581,10.849327,"[RESTROOM, SHOPPING, CAFE]"
3,B8GEWDTNZ8LXC,Alstad Kro og Camping,Alstad 1,None,68.272286,13.936212,"[RESTROOM, REST_STOP, RESTAURANT, ACCOMMODATIO..."
4,BA3I6G2IEG4CG,Altdorf,Prackenfelser Str. 8,None,49.381735,11.344057,"[RESTROOM, RESTAURANT, SHOPPING, WIFI, CAFE, C..."


In [4]:
# Different tariff definitions
#  - kr 5,39/kWh
#  - kr 5,39 / kWh
#  - kr 4,29 per kWh
#  - EUR 0.55 per kWh
#  - 5,49 SEK per kWh
#  - 4,29 kr/kWh + 0 kr/min
#  - kr 5,99/kWh. Over 80%: + kr 1/minutt

conn_cols = [c for c in raw_df.columns if c.startswith("connectionsTypes.")]

tariff_types = []
for _, row in raw_df.iterrows():
    station_id = row["id"]

    for col in conn_cols:
        conn_type = col.split(".", 1)[1]  # extract column type

        connectors = row[col] or []  # each is a list of dicts

        for item in connectors:
            tariff_types.append(
                {
                    "station_id": station_id,
                    "id": item["id"],
                    "connection_type": conn_type,
                    "tariff": item["tariffDefinition"]
                }
            )

tariff_df = pd.DataFrame(tariff_types)
tariff_df.head(5)

,station_id,id,connection_type,tariff
0,B8S7YSLRUBVUO,AGB1,AC Type 2,DKK 2.69 per kWh
1,B8S7YSLRUBVUO,AGB2,AC Type 2,DKK 2.69 per kWh
2,B8S7YSLRUBVUO,AGB3,AC Type 2,DKK 2.69 per kWh
3,B8S7YSLRUBVUO,AGB4,AC Type 2,DKK 2.69 per kWh
4,B9OXSNAF20Z5S,ABT1-A,CCS,"DKK 2,99 per kWh"


In [5]:
# In Norway, MVA = Merverdiavgift, which means Value-Added Tax (VAT).

# It is a 25% tax added to most goods and services.
# On EV charging invoices or tariffs, “inkl. MVA” means VAT included.
# “eks. MVA” means VAT excluded.

# Example:
# 5,39 kr/kWh inkl. MVA → final price.
# 5,39 kr/kWh eks. MVA → the station will add +25%.

# Lading for Brim Explorer -> Marine Charging
# Assumes AC and DC current
#   - AC = AC 4,29 kr/kWh
#   - AC = DC 5,39 Kr/kWh

tariff_df["tariff"].unique()

array(['DKK 2.69 per kWh', 'DKK 2,99 per kWh', 'kr 5,39/kWh',
       'kr 4,29 per kWh', 'kr 5,39 / kWh', 'kr 4,29/kWh',
       'EUR 0.55 per kWh', 'kr 5,99/kWh. Over 80%: + kr 1/minutt',
       '5,49 SEK per kWh', '3,39 SEK per kWh', 'kr 3.99 per kWh',
       'kr 3,99 per kWh', 'DC 5,39 kr/kWh + 1kr/min over 80%',
       'AC 4,29 kr/kWh', 'kr 2,5 per kWh',
       'Dynamic. Current price: kr 0,0 per kWh', 'kr 0 per kWh',
       'DC 5,39 Kr/kWh + 1kr/min over 80%', '4,29 kr/kWh + 0 kr/min',
       '4,0 kr/kWh + MVA, hurtiglading Lofoten',
       'kr 2,80 per kWh + MVA, Landstrøm Lofoten',
       'Lading for Brim Explorer'], dtype=object)

In [6]:
# From the analysis we can divide the tarifff column into 7 columns:
# - Price
# - Currency (ej: kr)
# - Measurement (ej: kWh)
# - Has extra tarifff (ej: Over 80%: + kr 1/minutt)
# - Has VAT (4,0 kr/kWh + MVA, hurtiglading Lofoten), default to false
# - VAT Location (ej: hurtiglading Lofoten)

# We can drop Marine Charging
# We can assume prices are dynamically changing.

# Doesn't have a number
extracted = tariff_df["tariff"].str.extract(r'(\d+[.,]?\d*)')[0]
tariff_df = tariff_df[extracted.notna()]

# Extract price and remove the ones that are 0
tariff_df["price"] = tariff_df["tariff"].str.extract(r'(\d+[.,]?\d*)')[0].str.replace(",", ".", regex=False).astype(float)
tariff_df = tariff_df[tariff_df["price"] != 0]

tariff_df["measurement"] = tariff_df["tariff"].str.extract(r'(kWh|MWh|Wh)')

In [7]:
def split_tariff(tariff_item: pd.Series):
    """
    Computes the required properties for a given tariff definition.

    Args:
        tariff_item (pd.Series): Current tariff definition.

    Returns:
        pd.Series: The computed values to add.
    """

    properties = {}
    tariff = tariff_item["tariff"]

    # Check all possible indexes or divisions (. and =)
    plus_index = tariff.find("+")
    new_line_index = tariff.find(". ")
    candidates = [i for i in [plus_index, new_line_index] if i != -1]

    # Which is closer to the start of the string?
    properties["tariff"] = tariff[0:min(candidates)] if candidates else tariff
    properties["extra_tariff"] = tariff[min(candidates)+1:].strip() if candidates else None

    return pd.Series(properties)

tariff_df[["tariff", "extra_tariff"]] = tariff_df.apply(split_tariff, axis=1)
tariff_df["currency"] = tariff_df["tariff"].str.extract(r'(DKK|kr|EUR|SEK)')

In [8]:
# Still need to check if it has MVA (tax)
tariff_df["extra_tariff"].unique()

array([None, 'Over 80%: + kr 1/minutt', '1kr/min over 80%', '0 kr/min',
       'MVA, hurtiglading Lofoten', 'MVA, Landstrøm Lofoten'],
      dtype=object)

In [9]:
def split_vat(tariff_item: pd.Series):
    """
    Computes the required properties for a given tariff definition.

    Args:
        tariff_item (pd.Series): Current tariff definition.

    Returns:
        pd.Series: The computed values to add.
    """

    properties = {}
    tariff = tariff_item["extra_tariff"]

    properties["has_vat"] = bool(tariff and "MVA" in tariff)
    properties["vat_location"] = tariff.split("MVA,")[1].strip() if properties["has_vat"] else None
    properties["extra_tariff"] = None if properties["has_vat"] else tariff

    return pd.Series(properties)


tariff_df[["has_vat", "vat_location", "extra_tariff"]] = tariff_df.apply(split_vat, axis=1)

In [10]:
tariff_df.head(10)

,station_id,id,connection_type,tariff,price,measurement,extra_tariff,currency,has_vat,vat_location
0,B8S7YSLRUBVUO,AGB1,AC Type 2,DKK 2.69 per kWh,2.69,kWh,None,DKK,False,None
1,B8S7YSLRUBVUO,AGB2,AC Type 2,DKK 2.69 per kWh,2.69,kWh,None,DKK,False,None
2,B8S7YSLRUBVUO,AGB3,AC Type 2,DKK 2.69 per kWh,2.69,kWh,None,DKK,False,None
3,B8S7YSLRUBVUO,AGB4,AC Type 2,DKK 2.69 per kWh,2.69,kWh,None,DKK,False,None
4,B9OXSNAF20Z5S,ABT1-A,CCS,"DKK 2,99 per kWh",2.99,kWh,None,DKK,False,None
5,B9OXSNAF20Z5S,ABT1-B,CCS,"DKK 2,99 per kWh",2.99,kWh,None,DKK,False,None
6,B9OXSNAF20Z5S,ABT2-A,CCS,"DKK 2,99 per kWh",2.99,kWh,None,DKK,False,None
7,B9OXSNAF20Z5S,ABT2-B,CCS,"DKK 2,99 per kWh",2.99,kWh,None,DKK,False,None
8,Alnasenter,AL1-1,CCS,"kr 5,39/kWh",5.39,kWh,None,kr,False,None
9,Alnasenter,AL1-3,CCS,"kr 5,39/kWh",5.39,kWh,None,kr,False,None


In [11]:
charging_data.head(5)

,id,name,address,description,latitud,longitud,amenities
0,B8S7YSLRUBVUO,Agerbæk Børnehus,Debelvej 25,Parkering ved børnehuset.,55.603490,8.811343,[]
1,B9OXSNAF20Z5S,Agerbæk Torv,Fåborgvej 177,None,55.600484,8.802200,"[SHOPPING, CAFE]"
2,Alnasenter,Alna Senter,Strømsveien 245,Lynladere i 1 etg utendørs parkeringshus ved M...,59.926581,10.849327,"[RESTROOM, SHOPPING, CAFE]"
3,B8GEWDTNZ8LXC,Alstad Kro og Camping,Alstad 1,None,68.272286,13.936212,"[RESTROOM, REST_STOP, RESTAURANT, ACCOMMODATIO..."
4,BA3I6G2IEG4CG,Altdorf,Prackenfelser Str. 8,None,49.381735,11.344057,"[RESTROOM, RESTAURANT, SHOPPING, WIFI, CAFE, C..."


In [12]:
# Add some random timestamps to simulate price history between 20 days ago and now
end = pd.Timestamp.now()
start = end - pd.Timedelta(days=20)

# total seconds in the range
total_seconds = int((end - start).total_seconds())

# generate random timestamps
random_ts = start + pd.to_timedelta(
    np.random.randint(0, total_seconds, size=len(tariff_df)),
    unit="s"
)

# convert to UNIX timestamp (seconds)
tariff_df["timestamp"] = random_ts.view("int64") // 10**9

In [13]:
tariff_df.head(10)

,station_id,id,connection_type,tariff,price,measurement,extra_tariff,currency,has_vat,vat_location,timestamp
0,B8S7YSLRUBVUO,AGB1,AC Type 2,DKK 2.69 per kWh,2.69,kWh,None,DKK,False,None,1764524942
1,B8S7YSLRUBVUO,AGB2,AC Type 2,DKK 2.69 per kWh,2.69,kWh,None,DKK,False,None,1763361325
2,B8S7YSLRUBVUO,AGB3,AC Type 2,DKK 2.69 per kWh,2.69,kWh,None,DKK,False,None,1763166049
3,B8S7YSLRUBVUO,AGB4,AC Type 2,DKK 2.69 per kWh,2.69,kWh,None,DKK,False,None,1764723976
4,B9OXSNAF20Z5S,ABT1-A,CCS,"DKK 2,99 per kWh",2.99,kWh,None,DKK,False,None,1763315028
5,B9OXSNAF20Z5S,ABT1-B,CCS,"DKK 2,99 per kWh",2.99,kWh,None,DKK,False,None,1764201552
6,B9OXSNAF20Z5S,ABT2-A,CCS,"DKK 2,99 per kWh",2.99,kWh,None,DKK,False,None,1764099127
7,B9OXSNAF20Z5S,ABT2-B,CCS,"DKK 2,99 per kWh",2.99,kWh,None,DKK,False,None,1764844287
8,Alnasenter,AL1-1,CCS,"kr 5,39/kWh",5.39,kWh,None,kr,False,None,1764692819
9,Alnasenter,AL1-3,CCS,"kr 5,39/kWh",5.39,kWh,None,kr,False,None,1763999168


In [64]:
filtered = charging_data[charging_data["id"].isin(tariff_df["station_id"].unique())]

print(f"Started with {charging_data["id"].nunique()} stations")
print(f"Left with {filtered["id"].nunique()} stations after removing invalid filters")


Started with 487 stations
Left with 482 stations


In [14]:
export_csv = tariff_df[["station_id", "connection_type", "id", "price", "measurement", "extra_tariff", "currency", "has_vat", "vat_location", "timestamp"]]

export_csv.to_csv("../assets/tariff_historical.csv", index=False, encoding="utf-8")


In [15]:
charging_data.to_csv("../assets/charging_stations.csv", index=False, encoding="utf-8")